In [5]:
pip install scikit-learn

  Using cached scikit_learn-1.7.1-cp310-cp310-win_amd64.whl.metadata (11 kB)
  Using cached scipy-1.15.3-cp310-cp310-win_amd64.whl.metadata (60 kB)
  Using cached joblib-1.5.1-py3-none-any.whl.metadata (5.6 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
Using cached scikit_learn-1.7.1-cp310-cp310-win_amd64.whl (8.9 MB)
Using cached joblib-1.5.1-py3-none-any.whl (307 kB)
Using cached scipy-1.15.3-cp310-cp310-win_amd64.whl (41.3 MB)
Using cached threadpoolctl-3.6.0-py3-none-any.whl (18 kB)

   ---------- ----------------------------- 1/4 [scipy]
   ---------- ----------------------------- 1/4 [scipy]
   ---------- ----------------------------- 1/4 [scipy]
   ---------- ----------------------------- 1/4 [scipy]
   ---------- ----------------------------- 1/4 [scipy]
   ---------- ----------------------------- 1/4 [scipy]
   ---------- ----------------------------- 1/4 [scipy]
   ---------- ----------------------------- 1/4 [scipy]
   ---------- ------------------

In [9]:
pip install streamlit

  Using cached blinker-1.9.0-py3-none-any.whl.metadata (1.6 kB)
   ---------------------------------------- 0.0/9.9 MB ? eta -:--:--
   - -------------------------------------- 0.3/9.9 MB ? eta -:--:--
   -- ------------------------------------- 0.5/9.9 MB 1.2 MB/s eta 0:00:08
   --- ------------------------------------ 0.8/9.9 MB 1.2 MB/s eta 0:00:08
   ---- ----------------------------------- 1.0/9.9 MB 1.2 MB/s eta 0:00:08
   ----- ---------------------------------- 1.3/9.9 MB 1.2 MB/s eta 0:00:07
   ------- -------------------------------- 1.8/9.9 MB 1.4 MB/s eta 0:00:06
   -------- ------------------------------- 2.1/9.9 MB 1.5 MB/s eta 0:00:06
   ---------- ----------------------------- 2.6/9.9 MB 1.6 MB/s eta 0:00:05
   ----------- ---------------------------- 2.9/9.9 MB 1.6 MB/s eta 0:00:05
   ------------- -------------------------- 3.4/9.9 MB 1.6 MB/s eta 0:00:05
   -------------- ------------------------- 3.7/9.9 MB 1.6 MB/s eta 0:00:05
   --------------- -------------------

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [1]:
import os
import xml.etree.ElementTree as ET
import pandas as pd

#  Change this to your MedQuAD base folder
base_dir = "C:\\Users\\abira\\OneDrive\\downloads\\MedQuAD-master\\MedQuAD-master"

all_data = []  # List to store all Q-A pairs

# Function to recursively extract full text from nested XML elements
def get_full_text(element):
    return ''.join(element.itertext()).strip() if element is not None else ""

# Loop through each _QA folder
for folder in os.listdir(base_dir):
    folder_path = os.path.join(base_dir, folder)
    if os.path.isdir(folder_path) and folder.endswith("_QA"):
        print(f"📁 Processing folder: {folder}")

        for file_name in os.listdir(folder_path):
            file_path = os.path.join(folder_path, file_name)
            if file_name.endswith(".xml") or file_name.isdigit():
                try:
                    tree = ET.parse(file_path)
                    root = tree.getroot()

                    # Get URL from the root tag
                    url = root.attrib.get('url', '')

                    # Loop through each QAPair
                    for qapair in root.iter():
                        if "qapair" in qapair.tag.lower():
                            q = a = ""
                            for child in qapair:
                                tag = child.tag.lower()
                                if "question" in tag:
                                    q = child.text.strip() if child.text else ""
                                elif "answer" in tag:
                                    a = get_full_text(child)  
                            if q: 
                                all_data.append({
                                    "Folder": folder,
                                    "File": file_name,
                                    "Question": q,
                                    "Answer": a,
                                    "URL": url
                                })

                except Exception as e:
                    print(f"⚠ Error in {folder}/{file_name}: {e}")

# Save combined CSV if data was extracted
if all_data:
    df_all = pd.DataFrame(all_data)
    output_path = os.path.join(base_dir, "All_MedQuAD_QA.csv")
    df_all.to_csv(output_path, index=False, encoding='utf-8')
    print(f"\n Combined CSV saved: {output_path}")
else:
    print("\n No data extracted from any folder.")

📁 Processing folder: 10_MPlus_ADAM_QA
📁 Processing folder: 11_MPlusDrugs_QA
📁 Processing folder: 12_MPlusHerbsSupplements_QA
📁 Processing folder: 1_CancerGov_QA
📁 Processing folder: 2_GARD_QA
📁 Processing folder: 3_GHR_QA
📁 Processing folder: 4_MPlus_Health_Topics_QA
📁 Processing folder: 5_NIDDK_QA
📁 Processing folder: 6_NINDS_QA
📁 Processing folder: 7_SeniorHealth_QA
📁 Processing folder: 9_CDC_QA

 Combined CSV saved: C:\Users\abira\OneDrive\downloads\MedQuAD-master\MedQuAD-master\All_MedQuAD_QA.csv


In [3]:
import pandas as pd
import re
import os


# Path to your dataset

file_path = r"C:\Users\abira\OneDrive\downloads\MedQuAD-master\MedQuAD-master\All_MedQuAD_QA.csv"


# Preprocessing function

def clean_text(text):
    if pd.isna(text):
        return ""
    text = text.lower()
    text = re.sub(r"<.*?>", "", text)  
    text = re.sub(r"\s+", " ", text)   
    return text.strip()


# Load dataset
df = pd.read_csv(file_path)

# Cleaning steps
# Drop rows with missing or empty Question/Answer
df = df.dropna(subset=["Question", "Answer"])
df = df[df["Question"].str.strip() != ""]
df = df[df["Answer"].str.strip() != ""]

# Apply cleaning
df["Question"] = df["Question"].apply(clean_text)
df["Answer"] = df["Answer"].apply(clean_text)

# Drop duplicates
df = df.drop_duplicates(subset=["Question", "Answer"]).reset_index(drop=True)


# Save cleaned dataset
dir_name = os.path.dirname(file_path)
save_path = os.path.join(dir_name, "All_MedQuAD_QA_Cleaned.csv")
df.to_csv(save_path, index=False, encoding="utf-8-sig")

print(f"✅ Cleaned dataset saved at: {save_path}")
print(f"Total rows after cleaning: {len(df)}")


✅ Cleaned dataset saved at: C:\Users\abira\OneDrive\downloads\MedQuAD-master\MedQuAD-master\All_MedQuAD_QA_Cleaned.csv
Total rows after cleaning: 15800


In [7]:
# app.py
import pandas as pd
import streamlit as st
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


# Load cleaned dataset

df = pd.read_csv(
    r"C:\Users\abira\OneDrive\downloads\MedQuAD-master\MedQuAD-master\All_MedQuAD_QA_Cleaned.csv"
)

# Extract questions and answers
questions = df["Question"].values
answers = df["Answer"].values


# Fit TF-IDF vectorizer

vectorizer = TfidfVectorizer(stop_words='english')
question_vectors = vectorizer.fit_transform(questions)


# Simple list of medical keywords for basic entity recognition
medical_keywords = [
    "asthma", "cancer", "diabetes", "infection", "fever", "headache", "surgery", "tumor",
    "symptom", "treatment", "medication", "disease", "nausea", "pain", "blood pressure"
]


# Streamlit UI

st.title("🩺 Medical Q&A Chatbot")
st.write("Ask any medical question and get reliable answers from the MedQuAD dataset.")

user_input = st.text_input("Enter your medical question:")

if user_input:
    # Entity recognition (basic)
    detected_entities = [word for word in medical_keywords if word in user_input.lower()]

    # Similarity-based retrieval
    user_vector = vectorizer.transform([user_input])
    similarities = cosine_similarity(user_vector, question_vectors).flatten()
    best_match_index = similarities.argmax()

    # Display best match
    st.subheader("🔍 Best Match:")
    st.markdown(f"**Q:** {questions[best_match_index]}")
    st.markdown(f"**A:** {answers[best_match_index]}")

    # Show detected medical entities
    if detected_entities:
        st.subheader("🧠 Detected Medical Entities:")
        st.write(", ".join(detected_entities))


2025-08-10 19:59:21.293 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-10 19:59:21.293 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-10 19:59:21.293 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-10 19:59:21.300 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-10 19:59:21.300 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-10 19:59:21.303 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-10 19:59:21.303 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-10 19:59:21.303 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [11]:
# to run in cmd 
# cd "C:\Users\abira\OneDrive\Downloads\MedQuAD-master\MedQuAD-master"
# streamlit run app.py


In [6]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import random


# Load dataset
df = pd.read_csv(r"C:\Users\abira\OneDrive\downloads\MedQuAD-master\MedQuAD-master\All_MedQuAD_QA_Cleaned.csv")


# Fit vectorizer

vectorizer = TfidfVectorizer(stop_words='english')
question_vectors = vectorizer.fit_transform(df["Question"])


# Function to get best match
def get_best_match(query):
    query_vector = vectorizer.transform([query])
    similarities = cosine_similarity(query_vector, question_vectors).flatten()
    best_index = similarities.argmax()
    return df.iloc[best_index]["Answer"]


# Accuracy Test
# Pick random 20 questions from dataset for testing

test_samples = df.sample(20, random_state=42)

correct = 0
for _, row in test_samples.iterrows():
    query = row["Question"]
    expected_answer = row["Answer"]
    predicted_answer = get_best_match(query)
    
    # Check exact match
    if predicted_answer.strip().lower() == expected_answer.strip().lower():
        correct += 1

accuracy = correct / len(test_samples)
print(f"Retrieval Accuracy: {accuracy*100:.2f}% ({correct}/{len(test_samples)})")


Retrieval Accuracy: 90.00% (18/20)
